<a href="https://colab.research.google.com/github/fatemafaria142/Exploration-of-Different-Prompting-Techniques-for-Automatic-Realistic-Story-Generation/blob/main/story_generation_zero_shot_prompt_using_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [4]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 4.9 MB/s eta 0:00:00


In [5]:
!pip install accelerate -U

# **Dataset Link:** https://huggingface.co/datasets/AtlasUnified/atlas-storyteller?row=42

In [6]:
from datasets import load_dataset

dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 5018
    })
})


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Get the first 5000 data points**

In [10]:
# Get the first 5000 data points
num_samples_to_display = 5000
subset_dataset = dataset['train'].select(range(num_samples_to_display))

# Display information for 3 data points from the subset
num_samples_to_show = 3
for i in range(num_samples_to_show):
    data = subset_dataset[i]
    print(f"Data Point {i + 1}:")
    print("ID:", data['id'])
    print("Input:", data['Story'])
    print("\n-----------------------------\n")

Data Point 1:
ID: seed_task_0
Input: In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room

# **Zero-Shot Story Generation Prompt**

In [13]:
def get_zero_shot_prompt(x):
    result = f"### Instruction:\nGenerate a story based on the following prompt:\n\n{x['Story']}"
    return result

# **Display prompts for the first 5 data points**

In [14]:
# Generate prompts for each data point in the subset dataset
prompts = []
for i in range(num_samples_to_display):
    data = subset_dataset[i]
    prompt = get_zero_shot_prompt(data)
    prompts.append(prompt)

# Display the generated prompts or use them as needed
for idx, prompt in enumerate(prompts[:5]):  # Display prompts for the first 5 data points
    print(f"Prompt for Data Point {idx + 1}:")
    print(prompt)
    print("\n-----------------------------\n")


Prompt for Data Point 1:
### Instruction:
Generate a story based on the following prompt:

In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made hi

# **More Examples of Prompts**

In [15]:
# Generate prompts for each data point in the subset dataset
prompts = []
for i in range(num_samples_to_display):
    data = subset_dataset[i]
    prompt = get_zero_shot_prompt(data)
    prompts.append(prompt)

# Display the generated prompts or use them as needed
for idx, prompt in enumerate(prompts[5:10]):  # Display prompts for the first 3 data points
    print(f"Prompt for Data Point {idx + 1}:")
    print(prompt)
    print("\n-----------------------------\n")


Prompt for Data Point 1:
### Instruction:
Generate a story based on the following prompt:

In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. Among this dystopian backdrop, two formidable gangs emerged as the most powerful contenders: the Razor Wings and the Shadow Serpents.  The city's unmistakable staleness hung in the air as the sun dipped below the horizon, casting long shadows across the cracked pavement. The Red Sector, a territory claimed by the Shadow Serpents, pulsated with an otherworldly energy. Led by the enigmatic and ruthless Viper, the gang thrived in the darkness, leaving a trail of chaos and blood in their wake.  In stark contrast, the Blue Sector was the bastion of the Razor Wings. Led by the charismatic yet relentless Phoenix, they were the embod

# **GPT2 and its tokenizer**
* https://huggingface.co/docs/transformers/model_doc/gpt2

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [18]:
# Set the padding token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# **Dataset Preparation and Tokenization for GPT-2 Training**
* This code segment appears to perform the following tasks:

* Assumes a dataset with specific keys ('instruction', 'input', 'output', and 'text').
* Iterates through the dataset to generate prompts based on the available data points.
* Tokenizes the generated prompts and output text using a tokenizer, preparing them as tensors.
* Compiles tokenized inputs, labels, and attention masks to be used for GPT-2 model training.

In [19]:
from transformers import Trainer, TrainingArguments
import torch
# Assuming subset_dataset is your dataset with 'id', 'Story'
dataset = subset_dataset

# Initialize empty lists to store inputs, targets, and attention masks
input_ids = []
labels = []
attention_masks = []

# Tokenize and prepare the dataset
for data_point in dataset:
    # Generate prompt
    prompt = get_zero_shot_prompt(data_point)

    # Tokenize prompt
    tokenized_prompts = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Tokenize output text (corrected text)
    tokenized_output = tokenizer(data_point['Story'], return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Append tokenized inputs, labels, and attention masks
    input_ids.append(tokenized_prompts['input_ids'])
    labels.append(tokenized_output['input_ids'])
    attention_masks.append(tokenized_prompts['attention_mask'])

# Convert lists to tensors
input_ids = torch.stack(input_ids)
labels = torch.stack(labels)
attention_masks = torch.stack(attention_masks)

### **Printing input_ids, labels, and attention_masks for the first 5 examples**

In [20]:
# Assuming the code provided earlier to prepare the dataset is already executed

# Printing input_ids, labels, and attention_masks for the first 5 examples
for i in range(3):
    print(f"Example {i+1}:")
    print("Input IDs:", input_ids[i])
    print("Labels:", labels[i])
    print("Attention Mask:", attention_masks[i])
    print("-----------------------")


Example 1:
Input IDs: tensor([[21017, 46486,    25,   198,  8645,   378,   257,  1621,  1912,   319,
           262,  1708,  6152,    25,   198,   198,   818,   262, 46609,  1748,
           286,   968,  1971,    11,   810,   262, 25988,  7588,   277,   677,
         28970,   290,   262,  2128,   286,  4979, 48134,   515,   832,   262,
          6483,    11,  5615,   257,   582,  3706, 28926, 18501,    13,  2080,
           465,   442,   271, 18449, 19218,  1370,    11, 30662,  4171,  2951,
            11,   290,   257, 48226, 23470,   416,   812,   286,  3047,    11,
         28926, 17273,   281,  1633,   286, 10715,   326, 33720,   883,   508,
         12606,   465,  3108,    13,   679,  5615,   257, 25565,  1204,   287,
           257, 12949,  7962,    11,  4581,   465,  1528,   355,   281, 42627,
            11,   465, 12513, 45853,   287, 16187,   290, 22780,    13,   220,
          1881, 46873,  3329,    11,   355,   262,  1748, 43363,   284,   262,
         14405,  9383,  4405, 

# **Dynamic Data Collation for GPT-2 Model Training**
* This code segment encompasses a class, GPT2DataCollator, which is designed to handle the collation of input features for GPT-2 model training. It dynamically checks the type of input features (whether they are dictionaries or tuples) and appropriately extracts input IDs, attention masks, and labels for padding and preparing the data to the same length. This data collation process is crucial for ensuring the uniformity and compatibility of the input features during the training of the GPT-2 model.

In [21]:
from transformers import Trainer, TrainingArguments
import torch


class GPT2DataCollator:
    def __call__(self, features):
        # Check if the features are dictionaries or tuples
        if isinstance(features[0], dict):
            input_ids = [feature['input_ids'] for feature in features]
            attention_masks = [feature['attention_mask'] for feature in features]
            labels = [feature['labels'] for feature in features]
        else:  # Assuming features are tuples
            input_ids = [feature[0] for feature in features]
            attention_masks = [feature[1] for feature in features]
            labels = [feature[2] for feature in features]

        # Pad inputs and labels to the same length
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'labels': labels
        }


# **Define the Training Arguments and Trainer**

In [24]:
'''
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-finetuned-version',    # Directory to save the model and checkpoints
    num_train_epochs=5,               # Number of training epochs
    per_device_train_batch_size=4,    # Batch size per device during training
    save_steps=5000,                  # Save checkpoint every X steps
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=500,                # Log training metrics every X steps
    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training
    report_to="none",                 # Disable evaluation during training
    prediction_loss_only=True,        # Compute only the prediction loss
    warmup_steps=500# number of warmup steps for learning rate scheduler
    # Add any additional arguments as needed
)

# Initialize the Trainer with the custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    data_collator=GPT2DataCollator()  # Use the custom data collator
)
'''

In [25]:
'''
# Start training
trainer.train()
'''

Epoch,Training Loss,Validation Loss
1,6.432100,6.214691
2,6.187800,6.050375
3,6.081100,5.942913
4,5.995300,5.874067
5,5.931500,5.841997


TrainOutput(global_step=6250, training_loss=6.17996515625, metrics={'train_runtime': 1367.5062, 'train_samples_per_second': 18.281, 'train_steps_per_second': 4.57, 'total_flos': 1633075200000000.0, 'train_loss': 6.17996515625, 'epoch': 5.0})

In [26]:
'''
trainer.save_model()
'''

# **Load the save model and train it again here...**

In [27]:
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = './gpt2-finetuned-version'  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-final-version',    # Directory to save the model and checkpoints
    num_train_epochs=5,               # Number of training epochs
    per_device_train_batch_size=4,    # Batch size per device during training
    save_steps=1000,                  # Save checkpoint every X steps
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=500,                # Log training metrics every X steps
    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training
    report_to="none",                 # Disable evaluation during training
    prediction_loss_only=True,        # Compute only the prediction loss
    warmup_steps=500# number of warmup steps for learning rate scheduler
    # Add any additional arguments as needed
)

# Initialize the Trainer with the custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    data_collator=GPT2DataCollator()  # Use the custom data collator
)

In [28]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.966000,5.816307
2,5.829100,5.672485
3,5.722800,5.558096
4,5.670200,5.487247
5,5.616300,5.453045


TrainOutput(global_step=6250, training_loss=5.74840458984375, metrics={'train_runtime': 1436.3619, 'train_samples_per_second': 17.405, 'train_steps_per_second': 4.351, 'total_flos': 1633075200000000.0, 'train_loss': 5.74840458984375, 'epoch': 5.0})

In [29]:
#Save the fine-tuned model here
trainer.save_model()

In [30]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels))
print("Perplexity:", eval_result['eval_loss'])

Perplexity: 5.45304536819458


# **Text Generation Using Fine-Tuned GPT-2 Model Pipeline**

In [38]:
from transformers import pipeline

# Define the generator pipeline using the fine-tuned GPT-2 model and tokenizer
generator = pipeline('text-generation', model='./gpt2-final-version', tokenizer='gpt2', pad_token_id=tokenizer.pad_token_id)

# Test prompt for generating text
test_prompt = (
    f"### Instruction:\nGenerate a story based on the following prompt:\n\nAmidst the inky blackness of the night "
)

# Generate text based on the test prompt using the generator pipeline
generated_test_text = generator(test_prompt, max_length=128, num_return_sequences=1)

# Print the generated text for the test prompt
print(generated_test_text[0]['generated_text'])



### Instruction:
Generate a story based on the following prompt:

Amidst the inky blackness of the night  of by It- a.anse in ofled. stood The It by, the the as... The.. its the within. of the. within the.... a In young As The of and His and in of with the Withiable It, beyond and of a the Born was the by and of eyes a was were his a his town The that their their was for whereled the of, and confines as that  curiosity for to to a to andled its. and


# **More Text Generation Examples**

In [36]:
from datasets import load_dataset

dataset = load_dataset("AtlasUnified/atlas-storyteller")

In [48]:
# Get the first 10 data points
num_samples_to_display = 10
subset_dataset = dataset['train'].select(range(num_samples_to_display))

# List to store 'Story' values
story_list = []

# Display information for 10 data points from the subset and save 'Story' values
num_samples_to_show = 10
for i in range(num_samples_to_show):
    data = subset_dataset[i]

    # Save 'Story' into the list
    story_list.append(data['Story'])

    print(f"Data Point {i + 1}:")
    print("ID:", data['id'])
    print("Story:", data['Story'])
    print("\n-----------------------------\n")

Data Point 1:
ID: seed_task_0
Story: In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room

In [53]:
from transformers import pipeline


# Define the generator pipeline using the fine-tuned GPT-2 model and tokenizer
generator = pipeline('text-generation', model='./gpt2-final-version', tokenizer='gpt2', pad_token_id=tokenizer.pad_token_id)

# List to store generated stories
generated_stories = []

# Generate text based on the subset of the training dataset
for example in subset_dataset:
    # Extract the first 7 words from example['Story']
    first_7_words = " ".join(example['Story'].split()[:7])

    # Create the test prompt using the extracted words
    test_prompt = (
        f"### Instruction:\nGenerate a story based on the following prompt:\n\n{first_7_words} "
    )

    # Generate text based on the test prompt using the generator pipeline
    generated_text = generator(test_prompt, max_length=128, num_return_sequences=1)

    # Extract and store only the generated story
    generated_story = generated_text[0]['generated_text'].split("### Instruction:\nGenerate a story based on the following prompt:\n\n")[1].strip()
    generated_stories.append(generated_story)

# Print or use the list of generated stories
for idx, story in enumerate(generated_stories):
    print(f"Generated Story {idx + 1}:\n{story}")
    print("------------------------------------------")


Generated Story 1:
In the bustling city of New York,  named, stood and It grand of the, towering architecture and and architecture intricate adorned architecture, with where its stood a architect renowned, to the of the intricate of a creativity, in - into a his creativity their the, and a, the sanctuary with in's to building the theirledled where known of skysc was renowned to a was of but by city, was of of city the story the architecture named by of and the namedled named the. was with. world Amid in of. of. inspiration world his
------------------------------------------
Generated Story 2:
The sky stretched out before them, an  this the, a the clouds hung through the against with, air the nest a the the sky the,, the of stood rolling a, a, was war the of stood the the where had and a chaos conflict was a a conflict was hadered to to. of was lives from who of the chaos in and in soldiers a the.,-, the on the  the war chaos from. war the the horrors soldier despair. piercing of, the 

# **BERTScore**
* https://huggingface.co/spaces/evaluate-metric/bertscore

In [42]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [44]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.4 MB/s eta 0:00:00


In [56]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.2 MB/s eta 0:00:00


In [50]:
print(story_list)

['In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan\'s blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment\'s hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room, concealed behind a bookshelf, 

In [54]:
print(generated_stories)

["In the bustling city of New York,  named, stood and It grand of the, towering architecture and and architecture intricate adorned architecture, with where its stood a architect renowned, to the of the intricate of a creativity, in - into a his creativity their the, and a, the sanctuary with in's to building the theirledled where known of skysc was renowned to a was of but by city, was of of city the story the architecture named by of and the namedled named the. was with. world Amid in of. of. inspiration world his", 'The sky stretched out before them, an  this the, a the clouds hung through the against with, air the nest a the the sky the,, the of stood rolling a, a, was war the of stood the the where had and a chaos conflict was a a conflict was hadered to to. of was lives from who of the chaos in and in soldiers a the.,-, the on the  the war chaos from. war the the horrors soldier despair. piercing of, the soldiers. to. and In of his With of a', 'In the midst of a bustling metropol

# **Partial match with the distilbert-base-uncased model:**

In [59]:
from evaluate import load
from bert_score import score as bert_score
from sacrebleu import corpus_bleu

# Load BERTScore model
bertscore = load("bertscore")

# Prepare data
predictions = generated_stories
references = story_list

# Calculate BERTScore
bert_results = bertscore.compute(predictions=predictions, references=story_list, model_type="distilbert-base-uncased")

# Print BERTScore results
print("BERTScore Results:")
print(bert_results)

# Load BLEU metrics
bleu_metric = load("bleu")

# Calculate Bilingual Evaluation Understudy (BLEU)
results_bleu = bleu_metric.compute(predictions=generated_stories, references=story_list)
print(f"\nBLEU Score: {results_bleu }\n")

BERTScore Results:
{'precision': [0.6970096826553345, 0.7138975858688354, 0.7101558446884155, 0.7055715322494507, 0.7093360424041748, 0.7241700887680054, 0.7070430517196655, 0.7062804102897644, 0.7122389674186707, 0.7180802226066589], 'recall': [0.6475098133087158, 0.664555549621582, 0.6575219631195068, 0.6577696800231934, 0.6542364358901978, 0.6878328323364258, 0.6524725556373596, 0.660372257232666, 0.6583566665649414, 0.6526166200637817], 'f1': [0.6713485717773438, 0.6883434653282166, 0.6828261017799377, 0.68083256483078, 0.6806730031967163, 0.7055339217185974, 0.6786625385284424, 0.6825552582740784, 0.684238612651825, 0.6837852001190186], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.36.2)'}

BLEU Score: {'bleu': 0.00023617025412962943, 'precisions': [0.7476007677543186, 0.1308139534883721, 0.057729941291585124, 0.04743083003952569], 'brevity_penalty': 0.0018462001599520977, 'length_ratio': 0.13708722536508355, 'translation_length': 1042, 'reference_lengt